In [1]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import os

In [2]:
input_base_data_path='./input_data/'
input_daily_data_path='./input_data/daily/'

town_base_data_filename = 'Covid19_Osaka_TownNumDataBase.xlsx'
area2_base_data_filename = 'Covid19_Osaka_Area2NumDataBase.xlsx'
data_sheet_name = 'Data'
header_sheet_name = 'Header'

area2_town_filename = 'OsakaArea2TownTransList.xlsx'
town_population_filename = 'OsakaTownPopulation.xlsx'
area2_output_filename = 'Covid19_Osaka_Area2NumPopuData.xlsx'
town_output_filename = 'Covid19_Osaka_TownNumPopuData.xlsx'

In [3]:
clmns_area2_base = ['日付', '地域', '日別',  '累計']
clmns_town_base = ['日付', '市町村', '日別', '累計']

clmns_area2_opt = ['日付', '地域', '日別', '週平均', '累計']
clmns_town_opt = ['日付', '市町村', '日別', '週平均', '累計']

clmns_area2_popu_opt = ['日付', '地域', '日別', '週平均', '週累積', '先週累積', '累計',
                        '日別人口10万人当たり', '週平均人口10万人当たり', 
                        '週累積人口10万人当たり', '先週累積人口10万人当たり',
                        '週累積前週増加率']
clmns_town_popu_opt = ['日付', '市町村', '日別', '週平均', '週累積', '先週累積', '累計', 
                       '日別人口10万人当たり', '週平均人口10万人当たり', 
                       '週累積人口10万人当たり', '先週累積人口10万人当たり',
                       '週累積前週増加率']
clmns_town_popu = ['市町村','人口']
clmns_area2_popu = ['地域','人口']

In [4]:
# DataFrameの最終行に要素を追加する
def append_item_to_df(df_source, lst_item):
    # 追加するDataFrameの行数(=最終行に追加するための行番号)を求める
    i_add_row = len(df_source)
    # df_area2_add_dataの最終行に追加する
    df_source.loc[i_add_row,:] = lst_item
    # 追加結果を返す
    return df_source

In [5]:
# 元データを読み込む
# 市町村
input_town_base_data_filepath = os.path.join(input_base_data_path,town_base_data_filename)
df_town_base_data = pd.read_excel(input_town_base_data_filepath, sheet_name=data_sheet_name)
# 地域
input_area2_base_data_filepath = os.path.join(input_base_data_path, area2_base_data_filename)
df_area2_base_data = pd.read_excel(input_area2_base_data_filepath, sheet_name=data_sheet_name)
# 前回保存したindexを削除する
df_town_base_data = df_town_base_data.drop(columns=df_town_base_data.columns[0])
df_area2_base_data = df_area2_base_data.drop(columns=df_area2_base_data.columns[0])

# 市町村と地域の一覧表を読み込む
df_area2_town = pd.read_excel(area2_town_filename)
df_area2_town = df_area2_town.dropna()
df_area2_town = df_area2_town.set_index('市町村')
# 市町村一覧を取得する
towns = df_area2_town.index.tolist()
# 地域の一覧を取得する
area2s = df_area2_town['地域'].unique().tolist()

In [6]:
# 追加するデータを読み込む
# 日別の追加するデータを格納するDataFrameを宣言する
df_add_data = pd.DataFrame(columns=clmns_town_base)
# 地域別の追加データを格納するDataFrameを宣言する
df_area2_add_data = pd.DataFrame(columns=clmns_area2_base)

# 日別データファイルの一覧を取得する
daily_files = os.listdir(input_daily_data_path)
# 日別データファイルを順番に追加する
for file in daily_files:
    base, ext = os.path.splitext(file)
    if ext == '.xlsx':
        #for debug
        print(file)        
        daily_filepath = os.path.join(input_daily_data_path, file)
        # 人数データを読込む
        df_daily_num_data = pd.read_excel(daily_filepath, sheet_name=data_sheet_name)
        # 列名「発生者数」を「日別」に変更する
        df_daily_num_data = df_daily_num_data.rename(columns={'発生者数': '日別'})
        # 日付データを読込む
        df_header = pd.read_excel(daily_filepath, sheet_name=header_sheet_name, index_col='Label')
        add_day = df_header.at['add_day', 'Value']
        # 人数データに日付データを紐付けする
        df_daily_num_data['日付'] = add_day
        # 日別データを追加する
        df_add_data = pd.concat([df_add_data, df_daily_num_data], ignore_index=True, axis=0)
        #df_add_data = df_add_data.append(df_daily_num_data, ignore_index=True)

        # 地域別の追加データの作業用のDataFrameを作成する
        df_area2_add_data_work = df_daily_num_data.copy()
        # 追加データの市町村に対応する地域を割付ける
        for town in towns:
            df_area2_add_data_work.loc[df_area2_add_data_work['市町村']==town,'地域'] = df_area2_town.at[town,'地域']
        # 市町村の列を削除する
        df_area2_add_data_work = df_area2_add_data_work.drop(columns='市町村')
        # 地域別に集計し直す
        for area2 in area2s:
            num_daily = df_area2_add_data_work.loc[df_area2_add_data_work['地域']==area2, '日別'].sum()
            num_total = df_area2_add_data_work.loc[df_area2_add_data_work['地域']==area2, '累計'].sum()
            # 追加する情報をListにまとめる
            lst_area2_infection = [add_day, area2, num_daily, num_total]
            # df_area2_add_dataの最終行に追加する
            df_area2_add_data = append_item_to_df(df_area2_add_data, lst_area2_infection)
            # 追加するDataFrameの行数(=最終行に追加するための行番号)を求める
            #i_add_row = len(df_area2_add_data)
            # df_area2_add_dataの最終行に追加する
            #df_area2_add_data.loc[i_add_row,:] = lst_area2_infection
            # 追加する情報をSeriesにまとめる
            #s_area2_infection = pd.Series(
            #                    [add_day, 
            #                    area2,
            #                    num_daily,
            #                    num_total],
            #                    index = df_area2_add_data.columns)
            # 作成したSeriesをdf_area2_add_dataに追加する
            #df_area2_add_data = pd.concat([df_area2_add_data, s_area2_infection], ignore_index=True, axis=0)
            #df_area2_add_data = df_area2_add_data.append(s_area2_infection, ignore_index=True)
        # 最後に全域を追加する
        num_daily = df_area2_add_data_work['日別'].sum()
        num_total = df_area2_add_data_work['累計'].sum()
        # 追加する情報をListにまとめる
        lst_area2_infection = [add_day, '全域', num_daily, num_total]
        # df_area2_add_dataの最終行に追加する
        df_area2_add_data = append_item_to_df(df_area2_add_data, lst_area2_infection)
        # 追加する情報をSeriesにまとめる
        #s_area2_infection = pd.Series(
        #                    [add_day, 
        #                    '全域',
        #                    num_daily,
        #                    num_total],
        #                    index = df_area2_add_data.columns)
        # 作成したSeriesをdf_area2_add_dataに追加する
        #df_area2_add_data = pd.concat([df_area2_add_data, s_area2_infection], ignore_index=True, axis=0)
        #df_area2_add_data = df_area2_add_data.append(s_area2_infection, ignore_index=True)

0813.xlsx


In [7]:
# 市町村の元データに追加する
df_town_base_data = pd.concat([df_town_base_data, df_add_data], ignore_index=True, axis=0)
#df_town_base_data = df_town_base_data.append(df_add_data, ignore_index=True)
# 日付と市町村でソートする
df_town_base_data = df_town_base_data.sort_values(['市町村', '日付'])
# 次回の更新用にExcelファイルに保存する
df_town_base_data.to_excel(town_base_data_filename, sheet_name=data_sheet_name)

# 地域の元データに追加する
df_area2_base_data = pd.concat([df_area2_base_data, df_area2_add_data], ignore_index=True, axis=0)
#df_area2_base_data = df_area2_base_data.append(df_area2_add_data, ignore_index=True)
# 日付と地域でソートする
df_area2_base_data = df_area2_base_data.sort_values(['地域', '日付'])
# 次回の更新用にExcelファイルに保存する
df_area2_base_data.to_excel(area2_base_data_filename, sheet_name=data_sheet_name)

In [8]:
# 市町村の人口を読込む
df_town_population = pd.read_excel(town_population_filename)
# 市町村名をインデックスとする
df_town_population = df_town_population.set_index('市町村')
# 地域の人口を作成する
# 地域の人口の作業用DataFrameを宣言する
df_area2_population_work = df_town_population.copy()
# 地域の人口を格納するDataFrameを宣言する
df_area2_population = pd.DataFrame(columns=clmns_area2_popu)

# 市町村のリストから「大阪府外」と「調査中」を除外する
towns_popu  = towns.copy()
towns_popu.remove('大阪府外')
towns_popu.remove('調査中')
# 「市町村」に対応する「地域」を割付ける
for town in towns_popu:
    df_area2_population_work.at[town, '地域' ] = df_area2_town.at[town,'地域']
# インデックスを「地域」に変更する
df_area2_population_work = df_area2_population_work.set_index('地域')
# 市町村の列を削除する
#インデックスを変更したため自動的に消える

# 地域のリストから「その他」を除外する
area2s_popu = area2s.copy()
area2s_popu.remove('その他')
# 地域別に集計し直す
for area2 in area2s_popu:
    num_population = df_area2_population_work.at[area2, '人口'].sum()
    # 追加する情報をListにまとめる
    lst_area2_populagion = [area2, num_population]
    # df_area2_populationの最終行に追加する
    df_area2_population = append_item_to_df(df_area2_population, lst_area2_populagion)
    # 追加する情報をSeriesにまとめる
    #s_area2_populagion = pd.Series(
    #                     [area2,
    #                      num_population],
    #                     index = df_area2_population.columns)
    # 作成したSeriesをdf_area2_populationに追加する
    #df_area2_population = pd.concat([df_area2_population, s_area2_populagion], ignore_index=True, axis=0)
    #df_area2_population = df_area2_population.append(s_area2_populagion, ignore_index=True)
# 最後に全域を追加する
num_population = df_area2_population_work['人口'].sum()
# 追加する情報をListにまとめる
lst_area2_populagion = ['全域', num_population]
# df_area2_populationの最終行に追加する
df_area2_population = append_item_to_df(df_area2_population, lst_area2_populagion)
# 追加する情報をSeriesにまとめる
#s_area2_populagion = pd.Series(
#                     ['全域',
#                      num_population],
#                     index = df_area2_population.columns)
# 作成したSeriesをdf_area2_populationに追加する
#df_area2_population = pd.concat([df_area2_population, s_area2_populagion], ignore_index=True, axis=0)
#df_area2_population = df_area2_population.append(s_area2_populagion, ignore_index=True)
# 地域名をインデックスとする
df_area2_population = df_area2_population.set_index('地域')

In [9]:
# 市町村の週平均データと
# 市町村の人口10万人当たりの日別と週累計データを作成する
df_towns_num = pd.DataFrame(columns=clmns_town_opt)
# 市町村毎に処理する
for town in towns:
    # 市町村単位でbase_dataを切出す
    df_town_data_wk = df_town_base_data[df_town_base_data['市町村']==town].copy()
    # 日別感染者数の７日間移動平均を計算して結果を追加する
    df_town_data_wk['週平均'] = df_town_data_wk['日別'].rolling(7).mean()
    # 7日間累積を計算して結果を追加する
    df_town_data_wk['週累積'] = df_town_data_wk['日別'].rolling(7).sum()
    # １週間前の7日間累積を計算して結果を追加する
    df_town_data_wk['先週累積'] = df_town_data_wk['週累積'].shift(7)
    # １週間前の7日間累積が０の場合nanにする
    df_town_data_wk.loc[ df_town_data_wk['先週累積']==0, '先週累積' ] = np.nan
    # 7日間累積の１週間前からの増加率を計算して結果を格納する
    df_town_data_wk['週累積前週増加率'] = df_town_data_wk['週累積'] / df_town_data_wk['先週累積']

    # 人口10万人当たりのデータを計算する場合
    if town in towns_popu:
        # 人口10万人当たりの日別感染者を計算して結果を追加する
        df_town_data_wk['日別人口10万人当たり'] = (df_town_data_wk['日別']
                                                *(100000/df_town_population.at[town,'人口']))
        # 人口10万人当たりの日別感染者数の７日間移動平均を計算して結果を追加する
        df_town_data_wk['週平均人口10万人当たり'] = df_town_data_wk['日別人口10万人当たり'].rolling(7).mean()
        # 人口10万人当たりの7日間累積を計算して結果を追加する
        df_town_data_wk['週累積人口10万人当たり'] = df_town_data_wk['日別人口10万人当たり'].rolling(7).sum()
        # 人口10万人当たりの１週間前の7日間累積を計算して結果を追加する
        df_town_data_wk['先週累積人口10万人当たり'] = df_town_data_wk['週累積人口10万人当たり'].shift(7)    
        # 人口10万人当たりの１週間前の7日間累積が０の場合nanにする
        df_town_data_wk.loc[ df_town_data_wk['先週累積人口10万人当たり']==0, '先週累積人口10万人当たり' ] = np.nan
    # 追加した結果を格納する
    df_towns_num = pd.concat([df_towns_num, df_town_data_wk], ignore_index=True, axis=0)
    #df_towns_num = df_towns_num.append(df_town_data_wk, ignore_index=True)
# 集計結果をExcelファイルに保存する
df_towns_num.to_excel(town_output_filename)

In [10]:
# 地域の週平均データと
# 市町村の人口10万人当たりの日別と週累計データを作成する
df_area2s_num = pd.DataFrame(columns=clmns_area2_opt)
# 地域に全域を追加する
area2s.append('全域')
area2s_popu.append('全域')
# 地域毎に処理する
for area2 in area2s:
    # 地域単位でbase_dataを切出す
    df_area2_data_wk = df_area2_base_data[df_area2_base_data['地域']==area2].copy()
    # 日別感染者数の７日間移動平均を計算して結果を追加する
    df_area2_data_wk['週平均'] = df_area2_data_wk['日別'].rolling(7).mean()
    # 7日間累積を計算して結果を追加する
    df_area2_data_wk['週累積'] = df_area2_data_wk['日別'].rolling(7).sum()
    # １週間前の7日間累積を計算して結果を追加する
    df_area2_data_wk['先週累積'] = df_area2_data_wk['週累積'].shift(7)    
    # １週間前の7日間累積が０の場合nanにする
    df_area2_data_wk.loc[ df_area2_data_wk['先週累積']==0, '先週累積' ] = np.nan
    # 7日間累積の１週間前からの増加率を計算して結果を格納する
    df_area2_data_wk['週累積前週増加率'] = df_area2_data_wk['週累積'] / df_area2_data_wk['先週累積']

    # 人口10万人当たりのデータを計算する場合
    if area2 in area2s_popu:
        # 人口10万人当たりの日別感染者を計算して結果を追加する
        df_area2_data_wk['日別人口10万人当たり'] = (df_area2_data_wk['日別']
                                                 *(100000/df_area2_population.at[area2,'人口']))
        # 人口10万人当たりの日別感染者数の７日間移動平均を計算して結果を追加する
        df_area2_data_wk['週平均人口10万人当たり'] = df_area2_data_wk['日別人口10万人当たり'].rolling(7).mean()
        # 人口10万人当たりの7日間累積を計算して結果を追加する
        df_area2_data_wk['週累積人口10万人当たり'] = df_area2_data_wk['日別人口10万人当たり'].rolling(7).sum()
        # 人口10万人当たりの１週間前の7日間累積を計算して結果を追加する
        df_area2_data_wk['先週累積人口10万人当たり'] = df_area2_data_wk['週累積人口10万人当たり'].shift(7)    
        # 人口10万人当たりの１週間前の7日間累積が０の場合nanにする
        df_area2_data_wk.loc[ df_area2_data_wk['先週累積人口10万人当たり']==0, '先週累積人口10万人当たり' ] = np.nan
    # 追加した結果を格納する
    df_area2s_num = pd.concat([df_area2s_num, df_area2_data_wk], ignore_index=True, axis=0)
    #df_area2s_num = df_area2s_num.append(df_area2_data_wk, ignore_index=True)
# 集計結果をExcelファイルに保存する
df_area2s_num.to_excel(area2_output_filename)

In [11]:
print('\rFinish !              ')

Finish !              
